In [1]:
import pandas as pd
import re
from sklearn.utils import resample
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split

In [9]:
train = pd.read_csv("C:/Users/shrut/OneDrive/Desktop/hate speech/train.csv")
print("Training set: ", train.columns, train.shape, len(train))

test = pd.read_csv("C:/Users/shrut/OneDrive/Desktop/hate speech/test.csv")
print("Testing set: ", test.columns, test.shape, len(test))

Training set:  Index(['id', 'label', 'tweet'], dtype='object') (31962, 3) 31962
Testing set:  Index(['id', 'tweet'], dtype='object') (17197, 2) 17197


In [11]:
def clean_text(df, text_field):
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))
    return df
test_clean = clean_text(test, "tweet")
train_clean = clean_text(train, "tweet")

In [13]:
train_majority = train_clean[train_clean.label == 0]
train_minority = train_clean[train_clean.label == 1]
train_minority_upsampled = resample(train_minority, 
                                 replace=True,    
                                 n_samples=len(train_majority),   
                                 random_state=123)
train_upsampled = pd.concat([train_minority_upsampled, train_majority])
train_upsampled['label'].value_counts()


label
1    29720
0    29720
Name: count, dtype: int64

In [15]:
pipeline_sgd = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf',  TfidfTransformer()),
    ('nb', SGDClassifier()),])

X_train, X_test, y_train, y_test = train_test_split(train_upsampled['tweet'],
                                                    train_upsampled['label'],random_state = 0)

In [17]:
model = pipeline_sgd.fit(X_train, y_train)
y_predict = model.predict(X_test)
from sklearn.metrics import f1_score
f1_score(y_test, y_predict)

0.9696